In [ ]:
import numpy as np
from math import exp
from typing import List

In [ ]:
class Neuron:
    def __init__(self, value, _children=(), _op='', label=''):
        self.value = value
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children);
        self._op = _op;
        self.label = label # for visualization

    def __repr__(self):
        stringVal = f"{self.value}";
        return stringVal

    def __add__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value + other.value, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value * other.value, (self, other), '*')
        def _backward():
            self.grad += other.value * out.grad
            other.grad += self.value * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Neuron(self.value**other, (self, ), f'**{other}')
        def _backward():
            self.grad += other * (self.value ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        return self * other**-1

    def exp(self, pos=1):
        x = self.value*pos
        out = Neuron(exp(x), (self, ), 'exp')
        def _backward():
            self.grad += out.value * out.grad * pos
        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other

    def __rsub__(self, other): # other - self
        return -(self - other)

    def __rmul__(self, other): # other * self
        return self * other

    def __rtruediv__(self, other): # other / self
        other = other if isinstance(other, Neuron) else Neuron(other)
        return other * self**-1
    
    def __lt__(self,other):
        if(isinstance(other,Neuron)):
            return self.value < other.value
        else:
            return self.value < other
        
    def __gt__(self,other):
        if(isinstance(other,Neuron)):
            return self.value > other.value
        else:
            return self.value > other
    
    def __le__(self,other):
        if(isinstance(other,Neuron)):
            return self.value <= other.value
        else:
            return self.value <= other
        
    def __ge__(self,other):
        if(isinstance(other,Neuron)):
            return self.value >= other.value
        else:
            return self.value >= other


    def backward(self):
        topology = []
        visited = set()
        def build_topology(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topology(child)
                topology.append(v)
        build_topology(self)

        self.grad = 1.0
        for node in reversed(topology):
            node._backward()

    def __float__(self):
        return float(self.value)
    
    def getGrad(self):
        return float(self.grad)

    def hello(self):
        print("hello")

In [ ]:
class LayerWrapper:
    def __init__(self,
                 layer_neuron_width:List[int],
                 activation_function:str="linear",
                 weight_initialization:str="uniform",
                 uniform_parameter: tuple = (0,10,0),
                 normal_parameter: tuple = (5,3,0)
                 ):
        self.layer_selector:List[Layers] = []
        self.layer_count = len(layer_neuron_width)
        
        Layers(
            wrapper=self,
            layer_neuron_width=layer_neuron_width,
            activation_function=activation_function,
            weight_initialization=weight_initialization,
            uniform_parameter=uniform_parameter,
            normal_parameter=normal_parameter
        )
    
    def feedforward(self):
        self.layer_selector[0].feedforward()

    def __getitem__(self, idx):
        return self.layer_selector[idx]
    
    def __repr__(self):
        return f"{self.layer_selector}"
    

class Layers:

    # Static Attribute, cuma buat validasi input

    valid_activation_function = {"linear","relu","sigmoid","hyperbolic_tangent","softmax"}
    valid_weight_initialization = {"zero","uniform","normal"}

    # START OF INITIALIZATION
    def __init__(self,
                 wrapper:LayerWrapper,                      # Wrapper ini buat ngepoint ke wrappernya
                 layer_neuron_width:list,                   # List of neuron width, elemen index ke-n menentukan lebar layer ke-n
                 activation_function:str = "linear",        # Harus ada di atas, kubuat defaultnya ini, dan rekursif layer selanjutnya sama
                 weight_initialization:str = "uniform",     # Sama kyk activation function
                 current_iter:int = 0,                      # jgn disentuh, ini buat nandain iterasinya, krn inisialisasi rekursif
                 uniform_parameter: tuple = (0,10,0),       # Hanya terpakai kalau activation_functionnya uniform
                 normal_parameter: tuple = (5,3,0),         # kyk di atas, tp normal
                 
                 ):
        
        # Cek validasi
        self.inputValidation(
            func_name=activation_function,
            weight_name=weight_initialization,
            layer_neuron_width=layer_neuron_width
            )
        
        self.wrapper        : LayerWrapper                      = wrapper
        self.current_iter   : int                               = current_iter
        self.neuron_count   : int                               = layer_neuron_width[current_iter]
        self.bias_weight    : Neuron                            = Neuron(0)
        self._neurons       : np.ndarray[Any, np.dtype[Neuron]] = self.neuronGenerator(np.zeros,(self.neuron_count))

        self.weight_initialization  : str    = weight_initialization
        self.uniform_parameter  : tuple = uniform_parameter
        self.normal_parameter   : tuple = normal_parameter

        self.activation_function    : str   = activation_function

        wrapper.layer_selector.append(self)
        self.next = (
            Layers(
                wrapper=wrapper,
                layer_neuron_width=layer_neuron_width,
                activation_function=activation_function,
                current_iter=current_iter + 1,
                weight_initialization=weight_initialization,
                uniform_parameter=uniform_parameter,
                normal_parameter=normal_parameter
                )
            if current_iter < len(layer_neuron_width) - 1
            else None
        )
        self.initWeight()
    # END OF INITIALIZATION

    # Buat ngeprint
    def __repr__(self):
        return f"{self._neurons}"
    
    # Kadang reflek Layer[0] dan dapet error, mending lgsg keluarin Neuron nya
    def __getitem__(self, idx):
        return self._neurons[idx]

    def __setitem__(self, idx, val):
        if (isinstance(val,Neuron)):
            self._neurons[idx] = val
        else:
            self._neurons[idx] = Neuron(val)

    # Kalau ada yg aku miss tambahin aja
    def inputValidation(self,
                        func_name:str,
                        weight_name:str,
                        layer_neuron_width:list
                        ):
        
        if func_name not in Layers.valid_activation_function:
            raise ValueError(f"Valid activation function name: {Layers.valid_activation_function}")
        
        if weight_name not in Layers.valid_weight_initialization:
            raise ValueError(f"Valid weight initialization: {Layers.valid_weight_initialization}")
        
        if len(layer_neuron_width) < 1:
            raise ValueError(f"layer_neuron_width must be a list with at least one positive integer.")
        
        if not all(isinstance(n, int) and n > 0 for n in layer_neuron_width):
            raise ValueError(f"All elements in layer_neuron_width must be a positive integer")

    # Init weightnya ngikut dari named parameter    
    def initWeight(self):
        if self.weight_initialization=="normal":
            mean        = self.normal_parameter[0]
            variance    = self.normal_parameter[1]
            seed        = self.normal_parameter[2]
            self.initWeightNormal(mean,variance,seed)

        elif self.weight_initialization=="uniform":
            low     = self.uniform_parameter[0]
            high    = self.uniform_parameter[1]
            seed    = self.uniform_parameter[2]
            self.initWeightUniform(low,high,seed)
            
        elif self.weight_initialization=="zero":
            self.initWeightZero()

    # Tiga function di bawah buat initialization, buat manual assign per layer
    def initWeightUniform(
            self,
            low     : float =   0,
            high    : float =   10,
            seed    : int   =   0
            ):
        
        if self.next != None:
            self.weight_initialization = "uniform"
            rng = np.random.default_rng(seed=seed)
            next_neuron_count = self.next.neuron_count
            self.weight = self.neuronGenerator(rng.uniform,low,high,size=(next_neuron_count,self.neuron_count))

    def initWeightNormal(
            self,
            mean        :   float   =   5,
            variance    :   float   =   3,
            seed        :   int     =   0
            ):
        if self.next != None:
            self.weight_initialization = "normal"
            rng = np.random.default_rng(seed=seed)
            next_neuron_count = self.next.neuron_count
            self.weight = self.neuronGenerator(rng.normal,loc=mean,scale=variance*variance,size=(next_neuron_count,self.neuron_count))

    def initWeightZero(
            self
        ):
        if self.next != None:
            self.weight_initialization = "zero"
            next_neuron_count = self.next.neuron_count
            self.weight = self.neuronGenerator(np.zeros,shape=(next_neuron_count,self.neuron_count))
        
    # Feedforward rekursif, biar ga tolol kyk sebelumnya, mohon maaf
    def feedforward(self):
        if self.next != None:
            self.next._neurons = np.dot(self._neurons,self.weight.T) + self.bias_weight
            self.next._neurons = self.activate()
            self.next.feedforward()

    # set Layer ini ngisi manual node nya, baru sadar ga guna jg sih
    # node hidden layer bakal kereplace jg pas feedforward
    def setLayerNeurons(
            self,
            neuron_list:List[float]
            ):
        self._neurons = self.neuronGenerator(np.array,(neuron_list))
        self.neuron_count = len(self._neurons)
        if self.current_iter>0:
            self.wrapper.layer_selector[self.current_iter-1].initWeight()
        self.initWeight()

    # Set lebar/jumlah neuron dari 1 layer
    # di bawah ada set weight karena weight depend on next hidden layer width
    def setLayerWidth(
            self,
            width: int = 1,
        ):
        self._neurons = self.neuronGenerator(np.zeros,(width))
        self.neuron_count = width
        if self.current_iter>0:
            self.wrapper.layer_selector[self.current_iter-1].initWeight()
        self.initWeight()

    def setBias(self, bias:float):
        self.bias_weight = Neuron(bias)

    # Masih sama dgn kode lama, buat fill in neuron di numpy
    def neuronGenerator(self,generator, *args, **kwargs):
        value = generator(*args, **kwargs)
        return np.vectorize(Neuron)(value)
    
    # ACTIVATION FUNCTION

    def activate(self):
        if self.activation_function=="linear":
            return self._neurons
        elif self.activation_function=="relu":
            return self.relu()
        elif self.activation_function=="sigmoid":
            return self.sigmoid()
        elif self.activation_function=="hyperbolic_tangent":
            return self.hyperbolicTangent()
        elif self.activation_function=="softmax":
            return self.softMax()

    def relu(self):
        reluTemp = np.vectorize(lambda x: x if x > 0 else Neuron(0))
        return reluTemp(self._neurons)
    
    def sigmoid(self):
        def sigmoidScalar(x:Neuron):
            return 1/(1+x.exp(-1))
        
        vectorized_sigmoid = np.vectorize(sigmoidScalar)
        return vectorized_sigmoid(self._neurons)
    
    def hyperbolicTangent(self):
        def tanhScalar(x:Neuron):
            return (x.exp()-x.exp(-1))/(x.exp()+x.exp(-1))
        
        vectorized_tanh = np.vectorize(tanhScalar)
        return vectorized_tanh(self._neurons)
    
    def softMax(self):
        exp = np.vectorize(lambda x: x.exp())
        divide = np.vectorize(lambda x,y: x/y)
        temp = exp(self._neurons)
        sumTemp = np.sum(temp)
        return divide(temp,sumTemp)



In [ ]:
FFNN = LayerWrapper(
    layer_neuron_width=[2,5,3],
    activation_function="linear",
    weight_initialization="uniform",
    uniform_parameter=(0,10,1)
)


In [ ]:
FFNN.layer_selector[0].setLayerNeurons([-11,1])
FFNN.layer_selector[1].setLayerWidth(3)
FFNN.layer_selector[2].setLayerWidth(1)

In [ ]:
print(FFNN.layer_selector[0])
print(FFNN.layer_selector[1])
print(FFNN.layer_selector[2])
print(FFNN.layer_selector[0].weight)
print(FFNN.layer_selector[1].weight)

In [ ]:
FFNN.feedforward()

In [ ]:
FFNN

In [ ]:
FFNN[2][0].backward()

In [ ]:
FFNN[1]
x = np.sum(FFNN[1]._neurons)
x._prev

In [ ]:
print(FFNN[0])


In [ ]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.value, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot



In [ ]:
def sigmoid_scalar(x:Neuron):
    return (1+x.exp(-1))**-1

In [ ]:
a = Neuron(2)
b = sigmoid_scalar(a)
b.backward()
a.grad

In [ ]:
draw_dot(b)

In [ ]:
def softMax(array):
        exp = np.vectorize(lambda x: x.exp())
        divide = np.vectorize(lambda x,y: x/y)
        temp = exp(array)
        sumTemp = np.sum(temp)
        return divide(temp,sumTemp)

In [ ]:
a = Neuron(1)
b = Neuron(3)
c = Neuron(5)
testArr = np.array([a,b,c])
out = softMax(testArr)

In [ ]:
out